# Preprocessing data

In [1]:
import json
import numpy as np
import csv 
import sys

dictCountries={
    "Alemania":"Germany", 
    "Austria":"Austria",
    "Bélgica":"Belgium",
    "Bulgaria":"Bulgaria",
    "Chipre":"Cyprus",
    "Croacia":"Croatia",
    "Dinamarca":"Denmark",
    "Eslovenia":"Slovenia",
    "Estonia":"Estonia",
    "Finlandia":"Finland",
    "Francia":"France",
    "Grecia":"Greece",
    "Holanda":"Holland",
    "Hungría":"Hungary",
    "Irlanda":"Ireland",
    "Italia":"Italy",
    "Letonia":"Latvia",
    "Lituania":"Lithuania",
    "Luxemburgo":"Luxembourg",
    "Malta":"Malta",
    "Polonia":"Poland",
    "Portugal":"Portugal",
    "Reino Unido":"United Kingdom",
    "República Checa":"Czech Rep.",
    "República Eslovaca":"Slovakia",
    "Rumanía":"Romania",
    "Suecia":"Sweden",
    "Federación de Rusia":"Russia",
    "Noruega":"Norway",
    "Serbia":"Serbia",
    "Suiza":"Switzerland",
    "Ucrania":"Ukraine"}

invdictCountries = {v: k for k, v in dictCountries.items()}

#Data from: Instuto Nacional de Estadística www.ine.es
f = open("./sources/info.txt", "r")
reader = csv.reader(f)

dic = {}
for row in reader:
    name=row[0]
    row[1]=row[1].replace(".","")
    row[2]=row[2].replace(".","")
    if row[1].isnumeric():
        ret2013=int(row[1])
    else:
        ret2013=0
    if row[2].isnumeric():
        ret2016=int(row[2])
    else:
        ret2016=0   
    if name in dictCountries:
        entry = {}
        entry['r2k13']= ret2013
        entry['r2k16']= ret2016
        dic[dictCountries[name]]=entry
        
f.close()

f = open("./sources/info.json", "w")
f.write(json.dumps(dic))
f.close()

# Creating the map

In [14]:
import geopandas as gpd
import json
from collections import OrderedDict

from shapely.geometry import Polygon, mapping
from bokeh.models import GeoJSONDataSource, LinearColorMapper, LogColorMapper,ColorBar,LogTicker, AdaptiveTicker
from bokeh.io import show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper
)
from bokeh.palettes import Viridis6 as palette
from bokeh.palettes import (Blues9, BrBG9, BuGn9, BuPu9, GnBu9, Greens9,
                            Greys9, OrRd9, Oranges9, PRGn9, PiYG9, PuBu9,
                            PuBuGn9, PuOr9, PuRd9, Purples9, RdBu9, RdGy9,
                            RdPu9, RdYlBu9, RdYlGn9, Reds9, Spectral9, YlGn9,
                            YlGnBu9, YlOrBr9, YlOrRd9)
from bokeh.plotting import figure
standard_palettes = OrderedDict([("Blues9", Blues9), ("BrBG9", BrBG9),
                                ("BuGn9", BuGn9), ("BuPu9", BuPu9),
                                ("GnBu9", GnBu9), ("Greens9", Greens9),
                                ("Greys9", Greys9), ("OrRd9", OrRd9),
                                ("Oranges9", Oranges9), ("PRGn9", PRGn9),
                                ("PiYG9", PiYG9), ("PuBu9", PuBu9),
                                ("PuBuGn9", PuBuGn9), ("PuOr9", PuOr9),
                                ("PuRd9", PuRd9), ("Purples9", Purples9),
                                ("RdBu9", RdBu9), ("RdGy9", RdGy9),
                                ("RdPu9", RdPu9), ("RdYlBu9", RdYlBu9),
                                ("RdYlGn9", RdYlGn9), ("Reds9", Reds9),
                                ("Spectral9", Spectral9), ("YlGn9", YlGn9),
                                ("YlGnBu9", YlGnBu9), ("YlOrBr9", YlOrBr9),
                                ("YlOrRd9", YlOrRd9)])



#obtain countries shapes
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
europe = (world.loc[world['continent'] == 'Europe'])

#Filling data into the geometry
europe_json = json.loads(europe.to_json())

for i in range(len(europe_json['features'])):
    count_name = europe_json['features'][i]['properties']['name']
    if count_name in invdictCountries:
        europe_json['features'][i]['properties']['rer'] = dic[count_name]['r2k16']
    else:
        europe_json['features'][i]['properties']['rer'] = 0
        
#Deleting Spain polygon here
del europe_json['features'][10]

#Loading Spain's shape
spain = (world.loc[world['name'] == 'Spain'])

geo_source = GeoJSONDataSource(geojson=json.dumps(europe_json))
spain_source = GeoJSONDataSource(geojson=spain.to_json())

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(plot_width=500, plot_height=500,
    title="Españoles Residentes en el Extranjero Retornados",
    tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    x_range=(-20,40), y_range=(35,80)
)

palette=standard_palettes.get("Greens9")

palette.reverse()

color_mapper = LinearColorMapper(palette=palette, low=100, high=3000)

p.patches('xs', 'ys', fill_alpha=0.7, name="europe",
          fill_color={'field': 'rer', 'transform': color_mapper}, 
          line_color='black', line_width=0.5, source=geo_source)

p.patches('xs', 'ys', fill_alpha=1.0, 
          fill_color='black', 
          line_color='black', line_width=0.5, source=spain_source)

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
#Just active with Europe countries but Spain
hover.names=["europe"]
hover.tooltips = [
    ("Name", "@name"),
    ("Retornados", "@rer")
    #("(Long, Lat)", "($x, $y)"),
]

color_bar = ColorBar(color_mapper=color_mapper, ticker=AdaptiveTicker(),
                     label_standoff=12, border_line_color="black", location=(0,0))

p.add_layout(color_bar, 'right')

show(p)